# Graph Visualization

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/notebooks/networkViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml
!pip install -q ipysigma

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256

In [2]:

def hash_text(text):
  """
  Hashes the given text using SHA256 and returns a short hex digest.

  Args:
      text: The text to hash.

  Returns:
      A short hex digest of the hashed text.
  """
  return sha256(text.encode()).hexdigest()[:8]

In [3]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [7]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType
WHERE {
  graph ?g {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?type ?target .
  }
}

"""

## Thought:  we need to mod the query to get the schema:identifier for Person and Organization and likely others
## and make these the ?source / ?target values.  So we have common shared PIDs that are expressing relations.



# FILTER((?sType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))
# FILTER((?tType) IN (schema:Person, schema:Organization, schema:Dataset, schema:Course, schema:Document))

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)


In [17]:
df.to_parquet('./output/bcodmo.parquet')
# df = pd.read_parquet('path/to/your/file.parquet')

In [13]:
df.head(10)

,source,type,target,sType,tType
0,https://gleaner.io/xid/genid/clsqo4rh2h49b33a25e0,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqo4rh2h49b33a25fg,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
1,https://gleaner.io/xid/genid/clsqp8bh2h49b33bh9kg,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqp8bh2h49b33bh9l0,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
2,https://gleaner.io/xid/genid/clsqmirh2h49b337uh8g,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqmirh2h49b337ugu0,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
3,https://gleaner.io/xid/genid/clsqoejh2h49b33afpc0,https://schema.org/about,https://www.bco-dmo.org/dataset/773727,https://schema.org/DataDownload,https://schema.org/Dataset
4,https://gleaner.io/xid/genid/clsqoejh2h49b33afpc0,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqoejh2h49b33afpeg,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
5,https://gleaner.io/xid/genid/clsqnsjh2h49b339n1ig,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqnsjh2h49b339n180,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
6,https://gleaner.io/xid/genid/clsqmj3h2h49b337v0k0,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqmj3h2h49b337v0lg,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
7,https://gleaner.io/xid/genid/clsqmo3h2h49b3385v70,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqmo3h2h49b3385uv0,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
8,https://gleaner.io/xid/genid/clsqnv3h2h49b339qcb0,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqnv3h2h49b339qcbg,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum
9,https://gleaner.io/xid/genid/clsqmqjh2h49b3389080,http://spdx.org/rdf/terms#checksum,https://gleaner.io/xid/genid/clsqmqjh2h49b3388vi0,https://schema.org/DataDownload,http://spdx.org/rdf/terms#Checksum


In [56]:
# Get unique sTypes
unique_values = df['sType'].unique()
print(f"Unique values in column: {unique_values}")


Unique values in column: ['https://schema.org/DataDownload' 'https://schema.org/OfferCatalog'
 'https://schema.org/Person' 'https://schema.org/FundingAgency'
 'https://schema.org/MonetaryGrant' 'https://schema.org/DataCatalog'
 'https://schema.org/Service' 'https://schema.org/ResearchProject'
 'https://schema.org/Event' 'https://schema.org/Dataset'
 'https://schema.org/Organization' 'https://schema.org/Place']


In [77]:
# Get unique tTypes
unique_values = df['tType'].unique()
print(f"Unique values in column: {unique_values}")

Unique values in column: ['http://spdx.org/rdf/terms#Checksum' 'https://schema.org/Dataset'
 'https://schema.org/DataCatalog' 'https://schema.org/Service'
 'https://schema.org/PropertyValue' 'https://schema.org/Organization'
 'https://schema.org/FundingAgency' 'https://schema.org/DigitalDocument'
 'https://schema.org/OfferCatalog' 'https://schema.org/DefinedTerm'
 'https://schema.org/ImageObject' 'https://schema.org/ContactPoint'
 'https://schema.org/PostalAddress' 'https://schema.org/SearchAction'
 'https://schema.org/Audience' 'https://schema.org/Place'
 'https://schema.org/Person' 'https://schema.org/DataDownload'
 'https://schema.org/MonetaryGrant' 'https://schema.org/ResearchProject'
 'https://schema.org/GeoShape']


In [68]:
# filter genid out for now
filtered_df = df[~df['source'].str.contains("genid") & ~df['target'].str.contains("genid")]

In [69]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15992 entries, 56789 to 249074
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  15992 non-null  object
 1   type    15992 non-null  object
 2   target  15992 non-null  object
 3   sType   15992 non-null  object
 4   tType   15992 non-null  object
dtypes: object(5)
memory usage: 749.6+ KB


In [72]:
# make a new dataframe where only the rows matching types of interest remain
sTypes = ['https://schema.org/Dataset',    'https://schema.org/Organization','https://schema.org/Person', 'https://schema.org/FundingAgency']
tTypes = ['https://schema.org/Dataset',   'https://schema.org/Organization','https://schema.org/Person', 'https://schema.org/FundingAgency']

# lookfor = ['https://schema.org/Dataset',  'https://schema.org/Organization', 'https://schema.org/Person', 'https://schema.org/FundingAgency']
fdftemp = filtered_df[df['sType'].isin(sTypes)]
fdf = fdftemp[df['tType'].isin(tTypes)]

<ipython-input-72-62365e768303>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fdftemp = filtered_df[df['sType'].isin(sTypes)]
<ipython-input-72-62365e768303>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fdf = fdftemp[df['tType'].isin(tTypes)]


In [73]:
fdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12978 entries, 78641 to 249074
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  12978 non-null  object
 1   type    12978 non-null  object
 2   target  12978 non-null  object
 3   sType   12978 non-null  object
 4   tType   12978 non-null  object
dtypes: object(5)
memory usage: 608.3+ KB


In [74]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for index, row in fdf.iterrows():
    n1 = g.add_node(row['source'])
    n1['type'] = row['sType']
    
    n2 = g.add_node(row['target'])
    n2['type'] = row['tType']

    e1 = g.add_edge(n1, n2)
    
fname = "./output/testGraphML.xml"
parser = GraphMLParser()
parser.write(g, fname)

In [75]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./output/testGraphML.xml")

In [76]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)



Sigma(nx.MultiGraph with 6,492 nodes and 12,978 edges)